In [1]:
### If at any point you're receiving the following TypeError message:
#### TypeError: Cannot interpret '<attribute 'dtype' of 'numpy.generic' objects>' as a data type

### Check the compatibility of your versions of numpy and pandas:
#print(np.__version__)
#print(pd.__version__)

### Run installers if necessary
##!pip install numpy==1.16.5 --user
##!pip install pandas==0.25.3 --user

In [2]:
### Import libraries and dependencies
### Since the environments created in class have specific libraries attached, some of the libraries have been disabled.
### Will have to look into how to accommodate everything.

#import panel as pn
#pn.extension('plotly')
#import plotly.express as px
import pandas as pd
#import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
from dotenv import load_dotenv
#import alpaca_trade_api as tradeapi
#from MCForecastTools import MCSimulation
import json, requests
from pandas.io.json import json_normalize

## Variables and Functions

In [3]:
### Defining function to loosely simulate live typing, or an NPC text box
import time
import sys
from random import randrange

### Variable to hold completion badges used for scene sequencing
badge_case = {'intro_badge','cards_badge','houses_badge','stocks_badge','crypto_badge'}

### Variable created to hold list of NPC text
npc_chatter = []

### Function is based off an example found in Stack Overflow
def chatter(npc_chatter):
    print()
    for c in npc_chatter:
        sys.stdout.write(c)
        sys.stdout.flush()
        secs = '0.0' + str(randrange(5,20,1))
        secs = float(secs)
        time.sleep(secs)
    print()
    time.sleep(1)

### 'Chatter' function test. If the line break (\n) is desired, double tab to align the separated lines
test_chatter = 'NPC: "This is just a test to see if your function functions,\n      and it\'s functioning just fine, friend!"'
npc_chatter.append(test_chatter)
chatter(test_chatter)


NPC: "This is just a test to see if your function functions,
      and it's functioning just fine, friend!"


# Scene 1: Introduction

In [4]:
### Introductory Scene ###
def intro_sequence():
    chatter('The_Good_Knight: "Hey there! You must be the new recruit. I\'m Scarlet, and I\'ll be showing you the ropes."')
    chatter('The_Good_Knight: "Before we get started, what should I call you?"')
    user_name = input('"Enter your name here so I can address you properly!" ')
    user_name.capitalize()

    chatter(f'The_Good_Knight: "Nice to meet you, {user_name}!"')
    chatter('The_Good_Knight: "Now, I know you have a bit of experience with Python, but you\'ll have to bear with me here."')
    chatter('The_Good_Knight: "Since it\'s our first time working together, I\'ll have to see what you can do."')
    chatter('The_Good_Knight: "Luckily, I know some people who could use your expertise. Let me just load up the modules and.."')
    time.sleep(1)

### Stage Selection ###
def stage_select():
    chatter('The_Good_Knight: "Alright! Go ahead and select whichever one you\'d like to check out."')
    
    chatter('The_Good_Knight: "Pokémon cards have been getting insanely popular as of late, and a youngster named Joey needs some help with finding the value of his favorite cards."')
    chatter('The_Good_Knight: "You have to admire that entrepreneurial spirit!')

    user_stage_select = input('"')

intro_sequence()

stage_select()


The_Good_Knight: "Hey there! You must be the new recruit. I'm Scarlet, and I'll be showing you the ropes."

The_Good_Knight: "Before we get started, what should I call you?"


# Pokémon TCG API

In [ ]:
### Setup for the Pokémon TCG API module

### Create var to hold the url to Pokemon TCG API
url_pkmn_io = 'https://api.pokemontcg.io/v2/cards'
print(url_pkmn_io)

### Add format specifier to the request url

json_pkmn_io = url_pkmn_io + '?format=json'
print(json_pkmn_io)

### Execute GET request and store response
requests_pkmn_io = requests.get(json_pkmn_io)
print(requests_pkmn_io)
print('\n', type(requests_pkmn_io))

### Get and store JSON content
content_pkmn_io = requests_pkmn_io.content
print(content_pkmn_io)
print('\n', type(content_pkmn_io))

### Format data as JSON
data_pkmn_io = requests_pkmn_io.json()

### Print JSON with added indents to check data
print(json.dumps(data_pkmn_io, indent=4))

### Gotten into the habit of checking the class type every step of the way, which has been pivotal in debugging in the past
print(f'\nClass type for data_pkmn_io: {type(data_pkmn_io)}')

### Test to call data from JSON. Output should be 'Ampharos'
data_pkmn_io['data'][0]['name']

In [ ]:
### Convert JSON content into a DataFrame series
series_pkmn_io = pd.read_json(content_pkmn_io)
series_pkmn_io = series_pkmn_io['data']
print(series_pkmn_io.head(), '\n')

### Flatten JSON data using normalize() function to convert into DataFrame
df_pkmn = json_normalize(series_pkmn_io)
df_pkmn.set_index('name', inplace=True)
print(f'Class type for df: {type(df_pkmn)}\n')
print(df_pkmn.columns, '\n')


### Select columns with market price data, as well as the set info for each card to differentiate between similarly-named cards
df_market = df_pkmn[df_pkmn.columns[df_pkmn.columns.str.contains('prices.*market|set.*id|name|series')]]
print(f'Class type for market_df: {type(df_market)}\n')
print(df_market.columns)

### Fill any unknown prices with 0
df_market = df_market.fillna(0)

### Rename columns and index
df_market.columns = ['Set ID','Set Name','Set Series','Holo','Reverse-Holo','Normal','1st Ed. Holo']
df_market.index.rename('Name',inplace=True)

df_market

0    {'id': 'pl1-1', 'name': 'Ampharos', 'supertype...
1    {'id': 'det1-1', 'name': 'Bulbasaur', 'superty...
2    {'id': 'mcd19-1', 'name': 'Caterpie', 'superty...
3    {'id': 'dv1-1', 'name': 'Dratini', 'supertype'...
4    {'id': 'xy5-1', 'name': 'Weedle', 'supertype':...
Name: data, dtype: object 

Class type for df: <class 'pandas.core.frame.DataFrame'>

Index(['id', 'supertype', 'subtypes', 'level', 'hp', 'types', 'evolvesFrom',
       'abilities', 'attacks', 'weaknesses', 'resistances', 'retreatCost',
       'convertedRetreatCost', 'number', 'artist', 'rarity',
       'nationalPokedexNumbers', 'set.id', 'set.name', 'set.series',
       'set.printedTotal', 'set.total', 'set.legalities.unlimited',
       'set.ptcgoCode', 'set.releaseDate', 'set.updatedAt',
       'set.images.symbol', 'set.images.logo', 'legalities.unlimited',
       'images.small', 'images.large', 'tcgplayer.url', 'tcgplayer.updatedAt',
       'tcgplayer.prices.holofoil.low', 'tcgplayer.prices.holofoil.mid',
      

,Set ID,Set Name,Set Series,Holo,Reverse-Holo,Normal,1st Ed. Holo
Name,,,,,,,
Ampharos,pl1,Platinum,Platinum,7.36,3.05,0.00,0.0
Bulbasaur,det1,Detective Pikachu,Sun & Moon,0.33,0.00,0.00,0.0
Caterpie,mcd19,McDonald's Collection 2019,Other,8.41,0.00,0.00,0.0
Dratini,dv1,Dragon Vault,Black & White,1.28,0.00,0.00,0.0
Weedle,xy5,Primal Clash,XY,0.00,0.28,0.11,0.0
...,...,...,...,...,...,...,...
Pachirisu,dpp,DP Black Star Promos,Diamond & Pearl,2.31,0.00,0.00,0.0
Cherubi,bw11,Legendary Treasures,Black & White,0.00,0.93,0.37,0.0
Mothim,pop7,POP Series 7,POP,1.64,0.00,0.54,0.0


In [ ]:
print(type(df_market))
print(type(df_market['Set Name']))
print()

market = df_market['Set Name'].unique()
print(type(market))
print()

print(market)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>

<class 'numpy.ndarray'>

['Platinum' 'Detective Pikachu' "McDonald's Collection 2019"
 'Dragon Vault' 'Primal Clash' 'Team Rocket Returns' 'Aquapolis' 'Dragon'
 'HS—Triumphant' 'Legend Maker' 'Base Set 2' 'Supreme Victors'
 'Pokémon Rumble' 'HeartGold & SoulSilver' 'Secret Wonders'
 'Delta Species' 'Gym Heroes' 'Fossil' 'Team Up' 'Flashfire'
 'POP Series 6' 'Steam Siege' 'Cosmic Eclipse' 'Crystal Guardians'
 "Champion's Path" 'Plasma Blast' 'Diamond & Pearl' 'Hidden Fates'
 'Emerald' 'POP Series 7' 'Gym Challenge' 'Ancient Origins'
 'Black & White' 'Holon Phantoms' 'Burning Shadows' 'Sun & Moon'
 'Call of Legends' 'Southern Islands' 'Guardians Rising'
 'Great Encounters' 'Phantom Forces' 'Boundaries Crossed'
 'Legendary Treasures' 'Base' 'Team Rocket' 'POP Series 5'
 'Dark Explorers' 'POP Series 3' 'Team Magma vs Team Aqua'
 'Next Destinies' 'Plasma Freeze' 'Deoxys' 'Ultra Prism' 'Fates Collide'
 'POP Series 1' 

In [ ]:
### Function for searching through Pokémon card dataframe
def search_pkmn():
    input_pkmn = input("What would you like to search for?")    # Prompt for user input
    user_search = df_market[df_market.index == input_pkmn]      # Search index
    return user_search                                          # Return results
    
search_pkmn()

,Set ID,Set Name,Set Series,Holo,Reverse-Holo,Normal,1st Ed. Holo
Name,,,,,,,
Ampharos,pl1,Platinum,Platinum,7.36,3.05,0.00,0.00
Ampharos,ecard2,Aquapolis,E-Card,140.93,0.00,0.00,0.00
Ampharos,dp3,Secret Wonders,Diamond & Pearl,11.67,7.99,0.00,0.00
Ampharos,pop7,POP Series 7,POP,6.89,0.00,2.99,0.00
Ampharos,neo3,Neo Revelation,Neo,43.99,0.00,0.00,149.99
Ampharos,ex10,Unseen Forces,EX,15.06,14.98,0.00,0.00


# Pokémon TCG Scene
#### To-do
- Write narrative
- After searching, store prices
- 